### Install requirements

In [1]:
# ! pip install praw

### Import libraries

In [2]:
import praw
import socket
import threading
import json

### Server configuration

In [3]:
class RedditAPIServer:
    def __init__(self, client_id, client_secret, user_agent, subreddit_name, host='localhost', port=9999):
        self.client_id = client_id
        self.client_secret = client_secret
        self.user_agent = user_agent
        self.subreddit_name = subreddit_name
        self.host = host
        self.port = port
        self.clients = []
        self.lock = threading.Lock()
        self.reddit = None
        self.server_sock = None

    def start(self):
        self._initialize_client()
        self._create_server_socket()
        self._start_streaming_thread()
        self._accept_clients()

    def _initialize_client(self):
        self.reddit = praw.Reddit(
            client_id=self.client_id,
            client_secret=self.client_secret,
            user_agent=self.user_agent
        )

    def _create_server_socket(self):
        self.server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_sock.bind((self.host, self.port))
        self.server_sock.listen()
        print(f'Server listening on {self.host}:{self.port}')

    def _start_streaming_thread(self):
        stream_thread = threading.Thread(target=self._stream_data)
        stream_thread.start()

    def _accept_clients(self):
        while True:
            client_sock, client_addr = self.server_sock.accept()
            client_thread = threading.Thread(
                target=self._handle_client, args=(client_sock, client_addr))
            client_thread.start()

    def stop(self):
        self.server_sock.close()
        self._disconnect_clients()

    def _disconnect_clients(self):
        for client in self.clients:
            client[0].close()
        self.clients = []

    def _stream_data(self):
        for submission in self.reddit.subreddit(self.subreddit_name).stream.submissions():
            message, metadata = self._get_submission_data(submission)
            data = {
                'message': message,
                'metadata': metadata
            }
            disconnected_clients = []
            self.lock.acquire()
            for client in self.clients:
                try:
                    self._send_data_to_client(client[0], data)
                except (socket.error, OSError):
                    print(f'Client disconnected: {client[1]}')
                    disconnected_clients.append(client)
            for client in disconnected_clients:
                self.clients.remove(client)
            self.lock.release()

    def _get_submission_data(self, submission):
        message = submission.title
        metadata = {
            'author_name': submission.author.name if submission.author else None,
            'date': submission.created_utc,
            'score': submission.score,
            'num_comments': submission.num_comments,
            'upvote_ratio': submission.upvote_ratio,
            'text': submission.selftext if submission.selftext else '',
            'subreddit_name': submission.subreddit.display_name if submission.subreddit else ''
        }
        return message, metadata

    def _send_data_to_client(self, client_sock, data):
        client_sock.getpeername()
        client_sock.sendall((json.dumps(data)+"\n").encode('utf-8'))

    def _handle_client(self, client_sock, client_addr):
        print(f'New client connected: {client_addr}')
        self.lock.acquire()
        self.clients.append((client_sock, client_addr))
        self.lock.release()

        try:
            while True:
                data = client_sock.recv(1024)
                if not data:
                    break
        except socket.error:
            pass
        finally:
            print(f'Client disconnected: {client_addr}')
            self.lock.acquire()
            self.clients.remove((client_sock, client_addr))
            client_sock.close()
            self.lock


### Run the server

In [4]:
CLIENT_ID = 't4ulpW-K0wmu6DCi5FuOIA'
CLIENT_SECRET = 'mJaM8HtkAsG8xJf9OrGep6bxQjfuhQ'
USER_AGENT = 'macos:my-reddit-streaming-sentiment-analysis-app:v1.0'
SUBREDDIT_NAME = 'all'

server = RedditAPIServer(CLIENT_ID, CLIENT_SECRET, USER_AGENT, SUBREDDIT_NAME)
server.start()


Server listening on localhost:9999
New client connected: ('127.0.0.1', 60244)


KeyboardInterrupt: 